# Experiment #3: Open vSwitch 4 Networks 2 Bridge Nodes 1 Shared Link

In this experiment we try to connect 4 networks with different hosts together with a bridge in the middle. We will be using Open vSwitch for the bridge, alternate solutions are Linux Bridge and Behavioral Model (BMv2).

## References
- [Open vSwitch Example](https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/complex_recipes/openvswitch/openvswitch.ipynb)
- [BMv4](https://github.com/p4lang/behavioral-model)

## TODO's

- Add Open vSwitch flow configuratation on ports and so on.
- Add tools on how to track packets in the bridge.
- measure of congestion.


## Topology


<img src="./fig/Testbed-openvswitch-3.png" width=90%>

## Code

In [9]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/alphani/.fabric_token.json
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sa2994_0000240697
Bastion Private Key File,/home/alphani/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/alphani/work/fabric_config/slice_key.pub
Slice Private Key File,/home/alphani/work/fabric_config/slice_key


In [10]:
slice_name= "Testbed-openvswitch-3"

sites = [site1,site2,site3,site4] = fablib.get_random_sites(count=4)
print(f"Sites: {sites}")

site_node_count = 2
bridge1_name = 'bridge1'
bridge2_name = 'bridge2'

Sites: ['PRIN', 'SRI', 'GPN', 'NEWY']


In [11]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [12]:
bridge1 = slice.add_node(name=bridge1_name, site=site3, cores=2, ram=8, disk=10, image=default_image, host=f"{site3.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')

bridge2 = slice.add_node(name=bridge2_name, site=site4, cores=2, ram=8, disk=10, image=default_image, host=f"{site4.lower()}-w1.fabric-testbed.net")
bridge2_nic1 = bridge2.add_component(model='NIC_Basic', name='nic_local_1')
bridge2_nic2 = bridge2.add_component(model='NIC_Basic', name='nic_local_2')
bridge2_nic3 = bridge2.add_component(model='NIC_Basic', name='nic_local_3')

### Connecting bridges nodes 

In [13]:
net = slice.add_l2network(name=f'net-br1-br2')
net.add_interface(bridge1.get_interfaces()[2])
net.add_interface(bridge2.get_interfaces()[2])

### Adding Nodes to bridge 1

In [14]:
for br_num in range(2):
    site = sites[br_num]
    print(f"Adding nodes to {site}-bridge{br_num+1}")
    for node_num in range(site_node_count):
        node_name = f"{site.lower()}-bridge{br_num+1}-{node_num+1}"
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image=default_image, host=f"{site.lower()}-w2.fabric-testbed.net")
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
        net = slice.add_l2network(name=f"net-br{br_num+1}-{node_num+1}")
        net.add_interface(iface)

        if br_num%2==0:
            net.add_interface(bridge1.get_interfaces()[node_num])
        else:
            net.add_interface(bridge2.get_interfaces()[node_num])

Adding nodes to PRIN-bridge1
Adding nodes to SRI-bridge2


In [15]:
slice.submit();


Retry: 9, Time: 285 sec


ID,823bd6f9-2767-47ec-8363-05986b7b8dc5
Name,Testbed-openvswitch-3
Lease Expiration (UTC),2025-01-12 06:39:13 +0000
Lease Start (UTC),2025-01-11 06:39:13 +0000
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
5e032499-58e8-4b51-9055-2f606a8a0609,bridge1,2,8,10,default_ubuntu_22,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe1a:2767,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe1a:2767,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
cc7ff57c-45ab-48c7-8077-fd9a4ff7096c,bridge2,2,8,10,default_ubuntu_22,qcow2,newy-w1.fabric-testbed.net,NEWY,ubuntu,2001:400:a100:3040:f816:3eff:fe76:b671,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fe76:b671,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
110f88ed-60b2-4cad-9018-e177f52dca4f,prin-bridge1-1,2,8,10,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe73:4106,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe73:4106,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
919ce062-c4a7-4dad-b679-900f01c67ec3,prin-bridge1-2,2,8,10,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:feb4:c4e,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:feb4:c4e,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
59a847ac-b0bd-4a1d-a79a-a6859bf1cf33,sri-bridge2-1,2,8,10,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.91,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@192.5.67.91,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
7a0d1bb8-a7ab-4721-a420-7144348f55c2,sri-bridge2-2,2,8,10,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.189,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@192.5.67.189,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
60f24181-7262-496b-8f12-14044e6f6cb1,net-br1-1,L2,L2STS,None,None,None,Active,
3aa47158-c4cb-42ca-956e-5c4c3b930906,net-br1-2,L2,L2STS,None,None,None,Active,
ddcf0bf7-f2f1-487e-89b6-5a2bd34e6818,net-br1-br2,L2,L2STS,None,None,None,Active,
69a2d500-6d55-4d20-83d1-af11404d697b,net-br2-1,L2,L2STS,None,None,None,Active,
7e73dd60-edd7-432e-bbdd-cc95b830de60,net-br2-2,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
bridge1-nic_local_1-p1,p1,bridge1,net-br1-1,100,config,,32:A2:17:D7:38:7B,enp9s0,enp9s0,fe80::30a2:17ff:fed7:387b,6,HundredGigE0/0/0/5
bridge1-nic_local_2-p1,p1,bridge1,net-br1-2,100,config,,2E:5E:F3:E9:1B:AE,enp8s0,enp8s0,fe80::2c5e:f3ff:fee9:1bae,6,HundredGigE0/0/0/5
bridge1-nic_local_3-p1,p1,bridge1,net-br1-br2,100,config,,2A:53:67:68:DA:B1,enp7s0,enp7s0,fe80::2853:67ff:fe68:dab1,6,HundredGigE0/0/0/5
bridge2-nic_local_3-p1,p1,bridge2,net-br2-1,100,config,,06:21:1C:D3:8B:F9,enp9s0,enp9s0,fe80::421:1cff:fed3:8bf9,4,HundredGigE0/0/0/5
bridge2-nic_local_1-p1,p1,bridge2,net-br2-2,100,config,,02:9C:D1:81:80:D5,enp7s0,enp7s0,fe80::9c:d1ff:fe81:80d5,4,HundredGigE0/0/0/5
bridge2-nic_local_2-p1,p1,bridge2,net-br1-br2,100,config,,06:1E:52:DC:0C:DB,enp8s0,enp8s0,fe80::41e:52ff:fedc:cdb,4,HundredGigE0/0/0/5
prin-bridge1-1-nic_local-p1,p1,prin-bridge1-1,net-br1-1,100,config,,0A:67:13:80:B7:6D,enp7s0,enp7s0,fe80::867:13ff:fe80:b76d,4,HundredGigE0/0/0/9
prin-bridge1-2-nic_local-p1,p1,prin-bridge1-2,net-br1-2,100,config,,0A:CE:97:3C:EC:AF,enp7s0,enp7s0,fe80::8ce:97ff:fe3c:ecaf,4,HundredGigE0/0/0/9
sri-bridge2-1-nic_local-p1,p1,sri-bridge2-1,net-br2-1,100,config,,02:87:0F:34:90:FD,enp7s0,enp7s0,fe80::87:fff:fe34:90fd,4,HundredGigE0/0/0/9
sri-bridge2-2-nic_local-p1,p1,sri-bridge2-2,net-br2-2,100,config,,06:5C:64:37:8B:68,enp7s0,enp7s0,fe80::45c:64ff:fe37:8b68,4,HundredGigE0/0/0/9



Time to print interfaces 314 seconds


# OVS 

In [16]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

done bridge: bridge1
done bridge: bridge2
Done


In [17]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   52:c0:62:0b:67:4c
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   52:c0:62:0b:67:4c
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp7s0     designated listening  2     128.1
  enp8s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



In [18]:
bridge2 = slice.get_node(name=bridge2_name)
stdout, stderr = bridge2.execute('sudo ovs-vsctl add-br br0')
for interface in bridge2.get_interfaces():
    stdout, stderr = bridge2.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge2.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge2.execute('sudo ifconfig br0 up')

print("Done")
stdout, stderr = bridge2.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
stdout, stderr = bridge2.execute('sudo ovs-appctl stp/show')


Done
---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   52:c0:62:0b:67:4c
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  root-port       enp8s0
  root-path-cost  2

Bridge ID:
  stp-priority  32768
  stp-system-id   fe:68:67:c8:ee:4b
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp8s0     root       listening  2     128.1
  enp7s0     designated listening  2     128.2
  enp9s0     designated listening  2     128.3



# Host Setup 

In [19]:
networks = [
    "10.10.10.1/24",
    "10.10.10.2/24",
    "10.10.10.3/24",
    "10.10.10.4/24"
]

for br_num in range(2):
    site = sites[br_num]
    for i in range(2):
        host = slice.get_node(name=f'{site.lower()}-bridge{br_num+1}-{i+1}')
        stdout, stderr = host.execute('sudo apt-get -y install net-tools', quiet=True)
        stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
        stdout, stderr = host.execute(f'sudo ip addr add {networks[i+2*br_num]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)


In [20]:
host1 = slice.get_node(name=f'{site1.lower()}-bridge1-1')
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

PING 10.10.10.2 (10.10.10.2) 56(84) bytes of data.
64 bytes from 10.10.10.2: icmp_seq=1 ttl=64 time=313 ms
64 bytes from 10.10.10.2: icmp_seq=2 ttl=64 time=156 ms
64 bytes from 10.10.10.2: icmp_seq=3 ttl=64 time=156 ms
64 bytes from 10.10.10.2: icmp_seq=4 ttl=64 time=156 ms
64 bytes from 10.10.10.2: icmp_seq=5 ttl=64 time=156 ms

--- 10.10.10.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 156.185/187.520/312.781/62.630 ms


In [21]:
stdout, stderr = host1.execute('ping 10.10.10.3 -c 5')

PING 10.10.10.3 (10.10.10.3) 56(84) bytes of data.
64 bytes from 10.10.10.3: icmp_seq=1 ttl=64 time=458 ms
64 bytes from 10.10.10.3: icmp_seq=2 ttl=64 time=229 ms
64 bytes from 10.10.10.3: icmp_seq=3 ttl=64 time=229 ms
64 bytes from 10.10.10.3: icmp_seq=4 ttl=64 time=229 ms
64 bytes from 10.10.10.3: icmp_seq=5 ttl=64 time=229 ms

--- 10.10.10.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 228.736/274.666/458.239/91.786 ms


In [22]:
stdout, stderr = host1.execute('ping 10.10.10.4 -c 5')

PING 10.10.10.4 (10.10.10.4) 56(84) bytes of data.
64 bytes from 10.10.10.4: icmp_seq=1 ttl=64 time=458 ms
64 bytes from 10.10.10.4: icmp_seq=2 ttl=64 time=229 ms
64 bytes from 10.10.10.4: icmp_seq=3 ttl=64 time=229 ms
64 bytes from 10.10.10.4: icmp_seq=4 ttl=64 time=229 ms
64 bytes from 10.10.10.4: icmp_seq=5 ttl=64 time=229 ms

--- 10.10.10.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4002ms
rtt min/avg/max/mdev = 228.747/274.696/458.408/91.855 ms


In [ ]:
slice.delete()